In [ ]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
  -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2020-11-25 05:04:11--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.164.176, 172.217.15.112, 172.217.9.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.164.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   194MB/s    in 0.4s    

2020-11-25 05:04:12 (194 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
# 사전 훈련된 가중치 불러오기 
import os
from tensorflow.keras import layers 
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
# tensorflow.keras.applications 모듈은 사전 훈련된 가중치를 갖고 있는 다양한 신경망 구조를 포함
#inception_v3 모듈로부터 불러온 inceptionV3 함수를 사용해서 Inceptionv3 모델을 구성

In [ ]:
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape=(150,150,3),
                                include_top = False,
                                weights = None)

pre_trained_model.load_weights(local_weights_file)
# load_weights() 메서드를 사용해서 미리 다운로드한 가중치를 InceptionV3 모델에 적용한다.
for layer in pre_trained_model.layers:
  layer.trainable = False

pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_470 (Conv2D)             (None, 74, 74, 32)   864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_470 (BatchN (None, 74, 74, 32)   96          conv2d_470[0][0]                 
__________________________________________________________________________________________________
activation_470 (Activation)     (None, 74, 74, 32)   0           batch_normalization_470[0][0]    
_______________________________________________________________________________________

In [ ]:
# 마지막 층 출력 확인하기 
last_layer = pre_trained_model.get_layer('mixed7')
print('마지막 층:',last_layer.output.shape)
last_output = last_layer.output

마지막 층: (None, 7, 7, 768)


In [ ]:
# 모델 구성 / 컴파일하기
from tensorflow.keras.optimizers import RMSprop
x = layers.Flatten()(last_output) # 마지막 층의 출력을 1차원으로 변환
x = layers.Dense(1024, activation='relu')(x) # Dense()는 완전 연결된 뉴런층을 추가한다. 
x = layers.Dense(1, activation = 'sigmoid')(x)

model =  Model(pre_trained_model.input,x)
# Model()클래스에 Inceptionv3 모델의 입력과 새롭게 구성한 뉴런층을 입력함으로써 새로운 모델을 만듬.
model.compile(optimizer=RMSprop(lr=0.0001),
            loss='binary_crossentropy',
            metrics=['accuracy'])

In [ ]:
!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
       -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '//tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

--2020-11-25 05:16:45--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.13.80, 172.217.15.80, 172.217.7.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.13.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   273MB/s    in 0.2s    

2020-11-25 05:16:45 (273 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define our example directories and files
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)


train_datagen = ImageDataGenerator(rescale=1./255.,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255.)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                  batch_size=20,
                                                  class_mode='binary',
                                                  target_size=(150, 150))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                      batch_size=20,
                                                      class_mode='binary',
                                                      target_size=(150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
# 모델 훈련하기 
history = model.fit(
  train_generator,
  validation_data=validation_generator,
  steps_per_epoch=100,
  epochs=20,
  validation_steps=50,
  verbose=2
)

Epoch 1/20
100/100 - 19s - loss: 0.3397 - accuracy: 0.8645 - val_loss: 0.1963 - val_accuracy: 0.9250
Epoch 2/20
100/100 - 18s - loss: 0.2042 - accuracy: 0.9165 - val_loss: 0.0920 - val_accuracy: 0.9680
Epoch 3/20
100/100 - 19s - loss: 0.1866 - accuracy: 0.9290 - val_loss: 0.1128 - val_accuracy: 0.9610
Epoch 4/20
100/100 - 19s - loss: 0.1869 - accuracy: 0.9275 - val_loss: 0.1126 - val_accuracy: 0.9580
Epoch 5/20
100/100 - 19s - loss: 0.1644 - accuracy: 0.9455 - val_loss: 0.1230 - val_accuracy: 0.9600
Epoch 6/20
100/100 - 19s - loss: 0.1657 - accuracy: 0.9440 - val_loss: 0.0986 - val_accuracy: 0.9620
Epoch 7/20
100/100 - 19s - loss: 0.1804 - accuracy: 0.9380 - val_loss: 0.1385 - val_accuracy: 0.9510
Epoch 8/20
100/100 - 18s - loss: 0.1281 - accuracy: 0.9495 - val_loss: 0.1057 - val_accuracy: 0.9630
Epoch 9/20
100/100 - 18s - loss: 0.1462 - accuracy: 0.9475 - val_loss: 0.1275 - val_accuracy: 0.9580
Epoch 10/20
100/100 - 18s - loss: 0.1250 - accuracy: 0.9525 - val_loss: 0.1478 - val_accura

In [ ]:
# 훈련 결과 확인하기 
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()